# Sentence Transformers

When working with both `TF-IDF` and `GloVe`, the focus was on the individual words within the Psalms. We were not getting the models to approach understanding the meaning of an entire psalm or a verse of a psalm. An example of this is 
> “The cat sat on the mat.” ≈ “A feline is resting on a rug.”  
> → **Sentence Transformers** give these two nearly identical vectors, unlike **TF-IDF** or **GloVe**.

We can look at other methods, such as `BERT`, to start to train a model to understand the meanings of text. Once we train and test out a **BERT** model, we can extend it to focus one sentences or verses by using **SBERT**.

**Source:** *Medium* - [Mastering BERT Model: Building it from Scratch with Pytorch](https://medium.com/data-and-beyond/complete-guide-to-building-bert-model-from-sratch-3e6562228891)


In [3]:
# Importing the needed libraries
import os
from transformers import BertTokenizer, BertModel
import pandas as pd
import numpy as np
import torch

In [4]:
# Importing the Psalms data

# Psalms (Bible & Psalter)
full_psalms = pd.read_csv("../../../data/csv/grouped_psalm.csv")


# Verses
psalms_verses = pd.read_csv("../../../data/csv/cleaned_psalm_verses.csv")

In [5]:
full_psalms.head()

,Unnamed: 0,tradition,text,psalm_num,verse,cleaned_verse
0,0,Orthodox,Bible,1,Blessed is the man Who walks not in the counse...,blessed man walk counsel ungodly stand way sin...
1,1,Orthodox,Bible,2,Why do the nations rage And the people meditat...,nation rage people meditate vain thing king ea...
2,2,Orthodox,Bible,3,A psalm by David when he fled from the face of...,psalm david fled face son absalom olord afflic...
3,3,Orthodox,Bible,4,For the End in psalms an ode by David You hear...,end psalm ode david heard icalled god righteou...
4,4,Orthodox,Bible,5,For the End concerning the inheritance a psalm...,end concerning inheritance psalm david give ea...


In [6]:
psalms_verses.head()

,tradition,text,psalm_num,verse_num,verse
0,Orthodox,Bible,1,1,Blessed is the man Who walks not in the counse...
1,Orthodox,Bible,1,2,But his will is in the law of the Lord And in ...
2,Orthodox,Bible,1,3,He shall be like a tree Planted by streams of ...
3,Orthodox,Bible,1,4,Not so are the ungodly not so But they are lik...
4,Orthodox,Bible,1,5,Therefore the ungodly shall not rise in the ju...


In [7]:
# Importing the clean `txt` files of the psalms.

# psalms_bert.py
from transformers import BertTokenizer, BertModel
import torch

# Load your text data
with open("../word_embeddings/corpus.txt", "r", encoding="utf-8") as f:
    psalms = [line.strip() for line in f if line.strip()]

print(f"Loaded {len(psalms)} psalms.")


Loaded 301 psalms.


In [8]:
import os

# Define the folder and file paths
data_dir = "../data"
verses_path = os.path.join(data_dir, "verses.txt")
verses_index_path = os.path.join(data_dir, "verses_index.txt")

# Ensure the directory exists
os.makedirs(data_dir, exist_ok=True)


# Building the text file above but as each line is a verse. 
with open("../data/verses.txt", "w", encoding="utf-8") as verses_file, \
     open("../data/verses_index.txt", "w", encoding="utf-8") as verse_index_file:
    
    for line_number, row in enumerate(psalms_verses.itertuples(index=False), start=1):
        # 1. Corpus: cleaned text, one Psalm per line
        verses_file.write(str(row.verse).strip().replace("\n", " ") + "\n")
    
        
        # 2. Index file: line number → Psalm ## + tradition
        verse_index_file.write(f"{line_number}\tPsalm {row.psalm_num}, Verse {row.verse_num}\t{row.text}\n")

We now have the data prepared in into a text file of psalms and a separate one of verses. We can now build a `BERT` encoder.

## Encoding the Psalms
Testing that the encoder works. 

In [9]:
import torch
import transformers

print("Torch version:", torch.__version__)
print("Transformers version:", transformers.__version__)


Torch version: 2.2.2
Transformers version: 4.57.1


After Confirmation of the package versions, lets test it out. I think I had problems with the differing kernels that I was working in. The following code is for testing from `Chat GPT`

In [10]:
from transformers import AutoTokenizer, AutoModel
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", use_fast=True)
model = AutoModel.from_pretrained("bert-base-uncased").to(device)
model.eval()

text = "Hello world"
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True).to(device)
with torch.no_grad():
    outputs = model(**inputs)
    embedding = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy()

print("Embedding shape:", embedding.shape)


Using device: cpu
Embedding shape: (768,)


With the Code above finally working, we can build the embeddings for the psalms and then for each of the verses. First we need to write the algorithm to do the encoding. 

In [11]:
# --- Clean Psalm Encoder using BERT ---
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np

# 1️⃣ Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# 2️⃣ Load tokenizer and model (fresh instances)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased", use_fast=True)
bert_model = AutoModel.from_pretrained("bert-base-uncased").to(device)
bert_model.eval()  # evaluation mode

# 4️⃣ Example usage: encode a list of psalms
# Make sure `psalms` is defined, e.g.
# psalms = ["Psalm 1 text...", "Psalm 2 text...", ...]



Using device: cpu


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [12]:
# 3️⃣ Encoding function with attention-mask weighted pooling
def encode_text_bert(text: str) -> np.ndarray:
    """
    Encode a single text string into a 1D numpy array (hidden_size,)
    Uses attention-mask weighted mean to ignore padding.
    """
    # Tokenize
    inputs = tokenizer(
        text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=512
    )
    
    # Move inputs to device
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = bert_model(**inputs)  # last_hidden_state: (1, seq_len, hidden)
        hidden = outputs.last_hidden_state
        mask = inputs.get("attention_mask")
        
        if mask is None:
            pooled = hidden.mean(dim=1)
        else:
            mask = mask.unsqueeze(-1)  # (1, seq_len, 1)
            masked_hidden = hidden * mask
            summed = masked_hidden.sum(dim=1)
            counts = mask.sum(dim=1).clamp(min=1e-9)
            pooled = summed / counts
    
    return pooled.squeeze(0).cpu().numpy()

In [13]:
'''
for i, psalm in enumerate(psalms, start=1):
    embedding = encode_text_bert(psalm)
    np.save(f"../data/bert/psalm_{i}_embedding.npy", embedding)
    if i % 10 == 0:
        print(f"Encoded {i} psalms.")

print("✅ All psalms encoded successfully.")'''

'\nfor i, psalm in enumerate(psalms, start=1):\n    embedding = encode_text_bert(psalm)\n    np.save(f"../data/bert/psalm_{i}_embedding.npy", embedding)\n    if i % 10 == 0:\n        print(f"Encoded {i} psalms.")\n\nprint("✅ All psalms encoded successfully.")'

In [14]:
import numpy as np
import os

output_dir = "../data/bert"
psalm_embeddings = []

# Load all saved embeddings
for filename in sorted(os.listdir(output_dir)):
    if filename.endswith(".npy") and "psalm_" in filename:
        emb = np.load(os.path.join(output_dir, filename))
        psalm_embeddings.append(emb)

psalm_embeddings = np.stack(psalm_embeddings)  # shape: (num_psalms, 768)
print("Loaded psalm embeddings:", psalm_embeddings.shape)


Loaded psalm embeddings: (301, 768)


In [15]:
def cosine_similarity(a: np.ndarray, b: np.ndarray) -> float:
    """Compute cosine similarity between two 1D numpy arrays."""
    a_norm = a / np.linalg.norm(a)
    b_norm = b / np.linalg.norm(b)
    return float(np.dot(a_norm, b_norm))

In [47]:
def log_top_matches(query, model_name, top_indices, similarities, full_psalms, filename="query_results.txt"):
    """
    Write the top psalm matches to a results file (and print them).
    Includes the original query and model information as a heading.
    """
    output_lines = []
    output_lines.append("=" * 80)
    output_lines.append(f"Query: {query}")
    output_lines.append(f"Model: {model_name}")
    output_lines.append("Top 5 matching psalms:")

    line_length = 125

    for rank, idx in enumerate(top_indices, start=1):
        text_type = "Bible" if idx <= 151 else "Psalter"
        num = idx if idx <= 151 else idx - 151
        psalm = full_psalms.iloc[idx]['verse']
        formatted_verse = ""
        # split into lines of length `line_length`
        for i in range(0, len(text), line_length):
            formatted_verse += psalm[i:i+line_length] + "\n"

        line = (
            f"{rank}. {text_type} Psalm {num + 1} - Similarity: {similarities[idx]}%\n"
            f"{formatted_verse}\n"
        )
        output_lines.append(line)

    # Combine into one result block
    result_block = "\n".join(output_lines)

    # Print to console
    print(result_block)

    # Append to a file
    with open(filename, "a", encoding="utf-8") as f:
        f.write(result_block + "\n\n")



In [ ]:
def query_bert(query):

    print("Query: ", query)

    if not query:
        print("Empty query. Exiting.")
        return
    
    # 1. Encode query with BERT
    query_emb = encode_text_bert(query)

    # 2. Compute similarities
    similarities = []
    for psalm_emb in psalm_embeddings:
        sim = cosine_similarity(query_emb, psalm_emb)
        similarities.append(round(sim * 100, 2))

    # 3. Get top 5 indices
    top_indices = np.argsort(similarities)[-5:][::-1]

    # 4. Log matches (and write to file)
    log_top_matches(query,
        model_name="BERT",
        top_indices=top_indices,
        similarities=similarities,
        full_psalms=full_psalms,
    )

    '''print("\nTop 5 matching psalms:")
    for rank, idx in enumerate(top_indices, start=1):
        text = "Bible"
        if idx > 151:
            num = idx - 151
            text = "Psalter"
        else: 
            num = idx
        print(f"{rank}. {text} Psalm {num + 1} - Similarity: {similarities[idx]}% \n {full_psalms.iloc[idx]['verse']}")
'''    


Now that `BERT` is working, We can further extend it to apply `SBERT`. Similar to `Bert`, `SBERT` focuses more on sentences of documents compared to just individual words. 
I will start to take the embeddeing code and change it to use `SBERT`'s methods. 

In [23]:
from sentence_transformers import SentenceTransformer

# Use a pretrained SBERT model
sbert_model = SentenceTransformer('all-mpnet-base-v2')  # or any SBERT variant


'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: e81eea54-d28e-4606-96e6-5e5bd1c11cad)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-mpnet-base-v2/resolve/main/./modules.json
Retrying in 1s [Retry 1/5].


After reading some of the documentation for `SBERT`, there is code I had to write for `BERT` that is already being handled, such as, tokenization, attention masks, and pooling. Lets write the function. 

In [25]:
def encode_text_SBERT(text):
    return sbert_model.encode(text, convert_to_numpy=True)



In [22]:
# Encoding each full Psalm
for i, psalm in enumerate(full_psalms, start=1):
    try:
        embedding = encode_text_SBERT(psalm)
        np.save(f"../data/sbert/psalm_{i}_embedding.npy", embedding)
        if i % 10 == 0:
            print(f"Encoded {i} psalms.")
    except Exception as e:
        print(f"Failed at psalm {i}: {e}")
        


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Condensing the embedding in the format to reference them. 

In [ ]:
output_dir = "../data/sbert"
psalm_SBERT_embeddings = []

# Load all saved embeddings
for filename in sorted(os.listdir(output_dir)):
    if filename.endswith(".npy") and "psalm_" in filename:
        emb = np.load(os.path.join(output_dir, filename))
        psalm_SBERT_embeddings.append(emb)

psalm_SBERT_embeddings = np.stack(psalm_SBERT_embeddings)  # shape: (num_psalms, 768)
print("Loaded psalm embeddings:", psalm_SBERT_embeddings.shape)

Loaded psalm embeddings: (301, 768)


In [ ]:
def query_sbert(query):
    print("Query: ", query)

    if not query:
        print("Empty query. Exiting.")
        return
    
    query_emb = encode_text_SBERT(query)

    similarities = []

    for psalm_emb in psalm_SBERT_embeddings:
        sim = cosine_similarity(query_emb, psalm_emb)
        similarities.append(round(sim*100, 2))

    
    top_indices = np.argsort(similarities)[-5:][::-1]

    # Checking the Output
    # print(top_indices)


    # 4. Log matches (and write to file)
    log_top_matches(query,
        model_name="SBERT",
        top_indices=top_indices,
        similarities=similarities,
        full_psalms=full_psalms,
    )
    
    '''print("\nTop 5 matching psalms:")
    for rank, idx in enumerate(top_indices, start=1):
        text = "Bible"
        if idx > 151:
            num = idx - 151
            text = "Psalter"
        else: 
            num = idx
        print(f"{rank}. {text} Psalm {num + 1} - Similarity: {similarities[idx]}% \n {full_psalms.iloc[idx]['verse']}")'''
    




## Testing Both BERT & SBERT

In [ ]:
query = input("Enter something to search for:")

query_bert(query)
query_sbert(query)                                  


Query:  For the truth
Query: For the truth
Model: BERT
Top 5 matching psalms:
1. Psalter Psalm 34 - Similarity: 50.29%
Judge Thou, O Lord, them that do me injustice; fight against them that fight against me. Take hold of weapon and buckler, and


2. Bible Psalm 18 - Similarity: 49.81%
For the End a psalm by David The heavens declare the glory of God The firmament shows the creation of His hands Day to day ut


3. Psalter Psalm 42 - Similarity: 47.85%
Judge me, God, and give judgment in my cause against a nation that is not holy; deliver me from an unjust and deceitful man. 


4. Bible Psalm 71 - Similarity: 47.2%
For Solomon OGod give Your judgments to the King And Your righteousness to the Kings Son That He may judge Your people in rig


5. Bible Psalm 147 - Similarity: 47.16%
Alleluia of Aggeus and Zacharias Praise the Lord O Jerusalem Praise your God O Zion For He strengthens the bars of your gates


Query:  For the truth
Query: For the truth
Model: SBERT
Top 5 matching psalms:
1. P

## Encoding Psalm Verses

With the encoding and the querying done for each full Psalm, lets try to do the same for the individual verses. 

In [ ]:
psalms_verses

,tradition,text,psalm_num,verse_num,verse
0,Orthodox,Bible,1,1,Blessed is the man Who walks not in the counse...
1,Orthodox,Bible,1,2,But his will is in the law of the Lord And in ...
2,Orthodox,Bible,1,3,He shall be like a tree Planted by streams of ...
3,Orthodox,Bible,1,4,Not so are the ungodly not so But they are lik...
4,Orthodox,Bible,1,5,Therefore the ungodly shall not rise in the ju...
...,...,...,...,...,...
5000,Orthodox,Psalter,150,64,"Butter of kine, and milk of sheep, with fat of..."
5001,Orthodox,Psalter,150,65,"So Jacob ate, and was filled; and the beloved ..."
5002,Orthodox,Psalter,150,66,"They provoked Me to anger with strange gods, a..."
5003,Orthodox,Psalter,150,67,"They sacrificed unto demons, not to God; to go..."


Trying to figure out how to properly index the data frame to encode the Psalms Verses

In [ ]:
# Access a specific verse directly
print(psalms_verses.iloc[3]['verse'])


# Loop through the DataFrame and stop at the 5th row
for idx, row in enumerate(psalms_verses.itertuples()):
    if idx == 5:
        break
    print(row.verse)


Not so are the ungodly not so But they are like the dust the wind drives from the face of the earth
Blessed is the man Who walks not in the counsel of the ungodly Nor stands in the way of sinners Nor sits in the seat of the troublesome
But his will is in the law of the Lord And in His law he meditates day and night
He shall be like a tree Planted by streams of waters That produces its fruit in its season And his leaf shall not wither And whatever he does shall prosper
Not so are the ungodly not so But they are like the dust the wind drives from the face of the earth
Therefore the ungodly shall not rise in the judgment Nor sinners in the counsel of the righteous


In [ ]:

for idx, row in enumerate(psalms_verses.itertuples()):
    embedding = encode_text_bert(row.verse)
    np.save(f"../data/bert_verses/psalm{row.psalm_num}:{row.verse_num}_embedding.npy", embedding)
    if idx % 10 == 0:
        print(f"Encoded {idx} psalms")

print("✅ All psalms encoded successfully.")

Encoded 0 psalms
Encoded 10 psalms
Encoded 20 psalms
Encoded 30 psalms
Encoded 40 psalms
Encoded 50 psalms
Encoded 60 psalms
Encoded 70 psalms
Encoded 80 psalms
Encoded 90 psalms
Encoded 100 psalms
Encoded 110 psalms
Encoded 120 psalms
Encoded 130 psalms
Encoded 140 psalms
Encoded 150 psalms
Encoded 160 psalms
Encoded 170 psalms
Encoded 180 psalms
Encoded 190 psalms
Encoded 200 psalms
Encoded 210 psalms
Encoded 220 psalms
Encoded 230 psalms
Encoded 240 psalms
Encoded 250 psalms
Encoded 260 psalms
Encoded 270 psalms
Encoded 280 psalms
Encoded 290 psalms
Encoded 300 psalms
Encoded 310 psalms
Encoded 320 psalms
Encoded 330 psalms
Encoded 340 psalms
Encoded 350 psalms
Encoded 360 psalms
Encoded 370 psalms
Encoded 380 psalms
Encoded 390 psalms
Encoded 400 psalms
Encoded 410 psalms
Encoded 420 psalms
Encoded 430 psalms
Encoded 440 psalms
Encoded 450 psalms
Encoded 460 psalms
Encoded 470 psalms
Encoded 480 psalms
Encoded 490 psalms
Encoded 500 psalms
Encoded 510 psalms
Encoded 520 psalms
Enco

In [ ]:
for idx, row in enumerate(psalms_verses.itertuples()):
    embedding = encode_text_SBERT(row.verse)
    np.save(f"../data/sbert_verses/psalm{row.psalm_num}:{row.verse_num}_embedding.npy", embedding)
    if idx % 10 == 0:
        print(f"Encoded {idx} psalms")

Encoded 0 psalms
Encoded 10 psalms
Encoded 20 psalms
Encoded 30 psalms
Encoded 40 psalms
Encoded 50 psalms
Encoded 60 psalms
Encoded 70 psalms
Encoded 80 psalms
Encoded 90 psalms
Encoded 100 psalms
Encoded 110 psalms
Encoded 120 psalms
Encoded 130 psalms
Encoded 140 psalms
Encoded 150 psalms
Encoded 160 psalms
Encoded 170 psalms
Encoded 180 psalms
Encoded 190 psalms
Encoded 200 psalms
Encoded 210 psalms
Encoded 220 psalms
Encoded 230 psalms
Encoded 240 psalms
Encoded 250 psalms
Encoded 260 psalms
Encoded 270 psalms
Encoded 280 psalms
Encoded 290 psalms
Encoded 300 psalms
Encoded 310 psalms
Encoded 320 psalms
Encoded 330 psalms
Encoded 340 psalms
Encoded 350 psalms
Encoded 360 psalms
Encoded 370 psalms
Encoded 380 psalms
Encoded 390 psalms
Encoded 400 psalms
Encoded 410 psalms
Encoded 420 psalms
Encoded 430 psalms
Encoded 440 psalms
Encoded 450 psalms
Encoded 460 psalms
Encoded 470 psalms
Encoded 480 psalms
Encoded 490 psalms
Encoded 500 psalms
Encoded 510 psalms
Encoded 520 psalms
Enco

In [54]:
output_dir = "../data/sbert_verses"
verses_SBERT_embeddings = []

# Load all saved embeddings
for filename in sorted(os.listdir(output_dir)):
    if filename.endswith(".npy") and "psalm" in filename:
        emb = np.load(os.path.join(output_dir, filename))
        verses_SBERT_embeddings.append(emb)

verses_SBERT_embeddings = np.stack(verses_SBERT_embeddings)  # shape: (num_psalm_verses, 768)
print("Loaded psalm embeddings:", verses_SBERT_embeddings.shape)

Loaded psalm embeddings: (2668, 768)


In [55]:
output_dir = "../data/bert_verses"
verses_BERT_embeddings = []

# Load all saved embeddings
for filename in sorted(os.listdir(output_dir)):
    if filename.endswith(".npy") and "psalm" in filename:
        emb = np.load(os.path.join(output_dir, filename))
        verses_BERT_embeddings.append(emb)

verses_BERT_embeddings = np.stack(verses_BERT_embeddings)  # shape: (num_psalm_verses, 768)
print("Loaded psalm embeddings:", verses_BERT_embeddings.shape)

Loaded psalm embeddings: (2668, 768)


## Writing the algorithm for the queries

In [35]:
def query_verses(query, model_name):
    """
    Run a query using either BERT or SBERT embeddings
    and display / log the top matches.
    """

    print("Query:", query)

    if not query:
        print("Empty query. Exiting.")
        return

    # --- Select embeddings based on model ---
    if model_name == "BERT":
        query_emb = encode_text_bert(query)
        embeddings = verses_BERT_embeddings
    elif model_name == "SBERT":
        query_emb = encode_text_SBERT(query)
        embeddings = verses_SBERT_embeddings
    else:
        raise ValueError(f"Unknown model_name: {model_name}")

    # --- Compute cosine similarities ---
    similarities = [
        round(cosine_similarity(query_emb, verse_emb) * 100, 2)
        for verse_emb in embeddings
    ]

    # --- Top 5 ranked verse indices ---
    top_indices = np.argsort(similarities)[-5:][::-1]

    # --- Log results to file ---
    log_top_matches(
        query=query,
        model_name=model_name,
        top_indices=top_indices,
        similarities=similarities,
        full_psalms=psalms_verses  # correct dataframe
    )

    # --- Pretty print results to console ---
    print(f"\nTop 5 matching verses using {model_name}:")
    line_length = 100

    for rank, idx in enumerate(top_indices, start=1):

        if idx >= len(psalms_verses):
            print(f"{rank}. [Index {idx} OUT OF RANGE]\n")
            continue

        verse_info = psalms_verses.iloc[idx]
        verse = ""
        verse_text = verse_info['verse']

        # Break the verse into multiple lines of fixed width
        for i in range(0, len(verse_text), line_length):
            verse += verse_text[i:i+line_length] + "\n"

        print(
            f"{rank}. {verse_info['text']} Psalm {verse_info['psalm_num']}, Verse {verse_info['verse_num']} "
            f"- Similarity: {similarities[idx]}%\n{verse}"
        )


# Query: *For the Peace of the World*
Lets test out the Algorithms and compares their results against each other. 
## Bert Results
### Searching Verses

In [36]:
query = input("Enter something to search: ")

query_verses(query, "BERT")
query_verses(query, "SBERT")

Query: For the abundance of the fruits of the earth
Query: For the abundance of the fruits of the earth
Model: BERT
Top 5 matching psalms:
1. Bible Psalm 65 - Similarity: 84.2%
Bring an end to the wickedness of sinners And keep straight the righteous For God examines hearts and reins

2. Psalter Psalm 170 - Similarity: 83.8%
He will guide the gentle in judgment He will teach the gentle His ways

3. Psalter Psalm 159 - Similarity: 83.72%
Whois this King of glory The Lord strong and mighty The Lord powerful in battle

4. Bible Psalm 64 - Similarity: 83.28%
The Lord shall judge the people Judge me O Lord according to my righteousness And according to the innocence within me

5. Psalter Psalm 2378 - Similarity: 82.79%
Arise, O Lord, in Thine anger; lift up Thyself to the uttermost borders of Thine enemies.


Top 5 matching verses using BERT:
1. Bible Psalm 7, Verse 10 - Similarity: 84.2%
Bring an end to the wickedness of sinners And keep straight the righteous For God examines hearts an
d 

NameError: name 'verses_SBERT_embeddings' is not defined

### Searching Full Psalms

In [ ]:
query_bert()

Query:  for the peace of the world

Top 5 matching psalms:
1. Psalter Psalm 34 - Similarity: 59.61% 
 Judge Thou, O Lord, them that do me injustice; fight against them that fight against me. Take hold of weapon and buckler, and rise up for mine help. Draw out the sword, and stop the way against them that persecute me; say unto my soul, I am Thy salvation. Let them be ashamed and confounded that seek after my soul; let them be turned back and put to shame that devise evil against me. Let them become as chaff before the face of the wind, and let the angel of the Lord afflict them. Let their way be dark and slippery, and let the angel of the Lord pursue them. For without cause have they hid for me destruction in their snare; without cause have they reproached my soul. Let a snare come upon him unawares, and let the snare that he hath hid catch himself; and into that very snare let him fall. But my soul shall exult in the Lord; it shall delight in His salvation. All my bones shall say, Lor

## SBERT Results
### Searching Verses

In [ ]:
query_verses(query, "SBERT")

Query:  For the Peace of the World

Top 5 matching verses using SBERT:
1. Bible Psalm 60, Verse 1 - Similarity: 61.09% 
 For the End in hymns by David Hear my supplication O God Attend to my prayer

2. Bible Psalm 117, Verse 20 - Similarity: 56.42% 
 This is the gate of the Lord The righteous shall go in through it

3. Bible Psalm 85, Verse 10 - Similarity: 53.99% 
 For You are great doing wonders You alone are the great God

4. Bible Psalm 3, Verse 5 - Similarity: 53.11% 
 I cried to the Lord with my voice And He heard me from His holy hill Pause

5. Bible Psalm 9, Verse 30 - Similarity: 50.8% 
 He lies in ambush in a hiding place like a lion in his den He lies in wait toseize the poor man To s
eize the poor man to drag him away



### Searching Full Psalms

In [56]:
query = input("Enter something to search for:")
# BERT
query_bert(query)
query_verses(query, "BERT")

# SBERT
query_verses(query, "SBERT")
query_sbert(query)      

Query:  For the truth
Query: For the truth
Model: BERT
Top 5 matching psalms:
1. Psalter Psalm 34 - Similarity: 50.29%
Judge Thou, O Lord, them that do me injustice; fight against them that fight against me. Take hold of weapon and buckler, and


2. Bible Psalm 18 - Similarity: 49.81%
For the End a psalm by David The heavens declare the glory of God The firmament shows the creation of His hands Day to day ut


3. Psalter Psalm 42 - Similarity: 47.85%
Judge me, God, and give judgment in my cause against a nation that is not holy; deliver me from an unjust and deceitful man. 


4. Bible Psalm 71 - Similarity: 47.2%
For Solomon OGod give Your judgments to the King And Your righteousness to the Kings Son That He may judge Your people in rig


5. Bible Psalm 147 - Similarity: 47.16%
Alleluia of Aggeus and Zacharias Praise the Lord O Jerusalem Praise your God O Zion For He strengthens the bars of your gates


Query: For the truth
Query: For the truth
Model: BERT
Top 5 matching psalms:
1. Psa